# Multilabel Tomato Disease Classification
This notebook trains ResNet50, EfficientNet-B0, and DenseNet121 to classify tomato leaf images.

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

from sklearn.metrics import f1_score, accuracy_score


In [ ]:
# path to your CSV
csv_path = 'Variant-b(MultiLabel Classification)/Multi-Label dataset - with augmented.csv'

# read CSV, only keep dataset & filename + label columns
df = pd.read_csv(csv_path)

# drop the 'path' column since we'll construct it from dataset+filename
df = df.drop(columns=['path', 'SUM'], errors='ignore')

# fill NA with 0 and ensure numeric
label_cols = df.columns.drop(['dataset','filename'])
df[label_cols] = df[label_cols].fillna(0).astype(int)

# preview
print(df.head())

In [ ]:
# 3.1: Class distribution (how many images per label)
plt.figure(figsize=(10,4))
df[label_cols].sum().sort_values().plot.barh()
plt.title("Label frequency over entire dataset")
plt.xlabel("Number of images")
plt.tight_layout()
plt.show()

# 3.2: Co-occurrence heatmap
import seaborn as sns
plt.figure(figsize=(8,6))
corr = df[label_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Label co-occurrence correlation")
plt.show()


In [ ]:
class TomatoDataset(Dataset):
    def __init__(self, df, root_dir, split, transform=None):
        """
        df         : DataFrame with columns ['dataset','filename',...labels]
        root_dir   : e.g. 'Variant-b(MultiLabel Classification)'
        split      : one of 'train','val','test'
        transform  : torchvision transforms
        """
        self.transform = transform
        self.data = df[df['dataset'] == split].reset_index(drop=True)
        self.root = os.path.join(root_dir, split)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.root, row['filename'])
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        labels = torch.FloatTensor(row[label_cols].values)
        return img, labels


In [ ]:
# transforms
train_tfms = T.Compose([
    T.Resize((224,224)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tfms = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# create datasets
root = 'Variant-b(MultiLabel Classification)'
train_ds = TomatoDataset(df, root, 'train', transform=train_tfms)
val_ds   = TomatoDataset(df, root, 'val',   transform=val_tfms)
test_ds  = TomatoDataset(df, root, 'test',  transform=val_tfms)

# loaders
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=4)


In [ ]:
def get_model(name, num_labels):
    """
    Return a pretrained backbone with a final sigmoid layer for multi-label.
    name: one of 'resnet50', 'densenet121', 'efficientnet_b0'
    """
    if name == 'resnet50':
        m = models.resnet50(pretrained=True)
        in_f = m.fc.in_features
        m.fc = nn.Linear(in_f, num_labels)
    elif name == 'densenet121':
        m = models.densenet121(pretrained=True)
        in_f = m.classifier.in_features
        m.classifier = nn.Linear(in_f, num_labels)
    elif name == 'efficientnet_b0':
        m = models.efficientnet_b0(pretrained=True)
        in_f = m.classifier[1].in_features
        m.classifier[1] = nn.Linear(in_f, num_labels)
    else:
        raise ValueError("Unknown model")
    return m

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.BCEWithLogitsLoss()


In [ ]:
def train_one_epoch(model, loader, opt):
    model.train()
    running_loss = 0.
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        opt.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    total_loss = 0.
    all_targets, all_preds = [], []
    for imgs, labels in loader:
        imgs = imgs.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels.to(device))
        total_loss += loss.item() * imgs.size(0)
        probs = torch.sigmoid(logits).cpu().numpy()
        all_preds.append(probs)
        all_targets.append(labels.numpy())
    avg_loss = total_loss / len(loader.dataset)
    preds = np.vstack(all_preds) >= 0.5  # threshold
    targets = np.vstack(all_targets)
    f1 = f1_score(targets, preds, average='micro')
    return avg_loss, f1


In [ ]:
models_to_run = ['resnet50', 'densenet121', 'efficientnet_b0']
histories = {}

for name in models_to_run:
    print(f"\n=== Training {name} ===")
    model = get_model(name, len(label_cols)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    history = {'train_loss':[], 'val_loss':[], 'val_f1':[]}

    for epoch in range(1, 11):   # 10 epochs
        tl = train_one_epoch(model, train_loader, optimizer)
        vl, vf = validate(model, val_loader)
        history['train_loss'].append(tl)
        history['val_loss'].append(vl)
        history['val_f1'].append(vf)
        print(f"Epoch {epoch:02d}: train_loss={tl:.4f}, val_loss={vl:.4f}, val_f1={vf:.4f}")

    # save state
    torch.save(model.state_dict(), f"{name}_multilabel.pth")
    histories[name] = history


In [ ]:
# losses
plt.figure(figsize=(12,4))
for name,h in histories.items():
    plt.plot(h['train_loss'], label=f'{name} train')
    plt.plot(h['val_loss'],   label=f'{name} val', linestyle='--')
plt.title("Train vs Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# F1
plt.figure(figsize=(6,4))
for name,h in histories.items():
    plt.plot(h['val_f1'], label=name)
plt.title("Validation F1 over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Micro F1")
plt.legend()
plt.show()


In [ ]:
summary = []
for name,h in histories.items():
    summary.append({
        'model': name,
        'final_val_loss': h['val_loss'][-1],
        'best_val_f1': max(h['val_f1'])
    })
summary_df = pd.DataFrame(summary).sort_values('best_val_f1', ascending=False)
print(summary_df)
